In [1]:
import geopandas as gpd
import h5py
import numpy as np
import yaml

from datetime import datetime
from torch_geometric.data import Data

c:\Users\Carlo\Documents\School\Masters\NUS\gnn_flood_modeling\venv\Lib\site-packages\torch_geometric\typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
c:\Users\Carlo\Documents\School\Masters\NUS\gnn_flood_modeling\venv\Lib\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
c:\Users\Carlo\Documents\School\Masters\NUS\gnn_flood_modeling\venv\Lib\site-packages\torch_geometric\typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def convert_hecras_timedata(time_data: list[np.bytes_]) -> np.ndarray:
    TIME_STAMP_FORMAT = '%d%b%Y %H:%M:%S'
    time_series = []
    for tstep in time_data:
        time_str = tstep.decode('UTF-8')
        time_stamp = datetime.strptime(time_str, TIME_STAMP_FORMAT)
        time_series.append(time_stamp)

    return np.array(time_series)

def get_timedata(path: str) -> np.ndarray:
    with h5py.File(path, 'r') as hec:
        # Get time data
        time_data = hec['Results']['Unsteady']['Output']['Output Blocks']['Base Output']['Unsteady Time Series'] \
            ['Time Date Stamp']
        time = convert_hecras_timedata(time_data)

    return time

In [4]:
def load_config(path: str) -> dict:
    with open(path) as f:
        try:
            config = yaml.safe_load(f)
        except:
            print('Error loading config.yaml')
    return config

def get_nested_value(dict: dict, dict_path: str, separator: str = '.'):
    keys = dict_path.split(sep=separator)
    for key in keys:
        d = d[key]
    return d

In [5]:
CONFIG_PATH ='config.yaml'

config = load_config(CONFIG_PATH)

In [6]:
hec_result_path = config['dataset_parameters']['hec_result_path']
timesteps = get_timedata(hec_result_path)
print('Total time steps: ', len(timesteps))
print('Difference between time steps: ', timesteps[1] - timesteps[0])

Total time steps:  337
Difference between time steps:  0:30:00


In [7]:
nodes_shape_path = config['dataset_parameters']['nodes_shape_path']
nodes = gpd.read_file(nodes_shape_path)
nodes

,X,Y,CC_index,Elevation1,geometry
0,320218.064510,6.373543e+06,0,95.920998,POINT (320218.065 6373543.045)
1,321418.064510,6.373543e+06,1,79.204002,POINT (321418.065 6373543.045)
2,321618.064510,6.373543e+06,2,77.579002,POINT (321618.065 6373543.045)
3,321818.064510,6.373543e+06,3,78.764999,POINT (321818.065 6373543.045)
4,322018.064510,6.373543e+06,4,78.786003,POINT (322018.065 6373543.045)
...,...,...,...,...,...
1263,316218.064510,6.369925e+06,1263,203.024994,POINT (316218.065 6369925.168)
1264,321412.664578,6.367267e+06,1264,111.728996,POINT (321412.665 6367266.906)
1265,317573.278635,6.365196e+06,1265,200.156006,POINT (317573.279 6365195.599)
1266,318261.091417,6.364419e+06,1266,163.639008,POINT (318261.091 6364419.073)


In [8]:
edges_shape_path = config['dataset_parameters']['edges_shape_path']
edges = gpd.read_file(edges_shape_path)
edges

,link_index,from_node,to_node,length,frm_node_E,to_node_E,slope,geometry
0,0,10,11,200.000,168.376007,135.397003,-0.164895,"LINESTRING (319418.065 6373343.045, 319618.065..."
1,1,10,1126,240.787,168.376007,228.136002,0.248186,"LINESTRING (319418.065 6373343.045, 319423.053..."
2,2,10,24,200.000,168.376007,164.438004,-0.019690,"LINESTRING (319418.065 6373343.045, 319418.065..."
3,3,11,25,200.000,135.397003,141.511002,0.030570,"LINESTRING (319618.065 6373343.045, 319618.065..."
4,4,11,12,200.000,135.397003,122.338997,-0.065290,"LINESTRING (319618.065 6373343.045, 319818.065..."
...,...,...,...,...,...,...,...,...
2607,2607,215,1263,217.873,148.550003,203.024994,0.250031,"LINESTRING (316218.065 6370143.045, 316218.065..."
2608,2608,373,1264,208.966,111.313004,111.728996,0.001991,"LINESTRING (321218.065 6367343.045, 321412.665..."
2609,2609,418,1265,154.096,187.365997,200.156006,0.083000,"LINESTRING (317618.065 6365343.045, 317573.279..."
2610,2610,432,1266,131.225,173.673996,163.639008,-0.076472,"LINESTRING (318218.065 6364543.045, 318261.091..."


In [9]:
edge_index = edges[['from_node', 'to_node']].to_numpy()
print(edge_index.shape)
print(edge_index)

(2612, 2)
[[  10   11]
 [  10 1126]
 [  10   24]
 ...
 [ 418 1265]
 [ 432 1266]
 [ 433 1267]]


In [10]:
pos = nodes[['X', 'Y']].to_numpy()
print(pos.shape)
print(pos)

(1268, 2)
[[ 320218.06451013 6373543.04451157]
 [ 321418.06451013 6373543.04451157]
 [ 321618.06451013 6373543.04451157]
 ...
 [ 317573.2786347  6365195.59913723]
 [ 318261.0914168  6364419.07311044]
 [ 319598.32679715 6364549.21827362]]


In [11]:
# Node static features
with h5py.File(hec_result_path, 'r') as hec:
    area = np.array(hec['Geometry']['2D Flow Areas']['Perimeter 1']['Cells Surface Area'])
    roughness = np.array(hec['Geometry']['2D Flow Areas']['Perimeter 1']["Cells Center Manning's n"])

elevation = nodes['Elevation1']
print(area.shape, area.dtype)
print(roughness.shape, roughness.dtype)
print(elevation.shape, elevation.dtype)

sf_nodes = np.stack([area, roughness, elevation], axis=1)
sf_nodes.shape

(1268,) float32
(1268,) float32
(1268,) float64


(1268, 3)

In [12]:
# Node dynamic features
with h5py.File(hec_result_path, 'r') as hec:
    wl=np.array(hec['Results']['Unsteady']['Output']['Output Blocks']['Base Output'] \
                       ['Unsteady Time Series']['2D Flow Areas']['Perimeter 1']['Water Surface'])
print(wl.shape, wl.dtype)

df_nodes = wl
df_nodes.shape

(337, 1268) float32


(337, 1268)

In [13]:
# Edge static features
with h5py.File(hec_result_path, 'r') as hec:
    struct_feats = np.array(hec['Geometry']['2D Flow Areas']['Perimeter 1']['Faces NormalUnitVector and Length'])
    direction_x = struct_feats[:, 0]
    direction_y = struct_feats[:, 1]
    face_length = struct_feats[:, 2]

length = edges['length']
slope = edges['slope']

print(direction_x.shape, direction_x.dtype)
print(direction_y.shape, direction_y.dtype)
print(face_length.shape, face_length.dtype)
print(length.shape, length.dtype)
print(slope.shape, slope.dtype)

sf_edges = np.stack([direction_x, direction_y, face_length, length, slope], axis=1)
sf_edges.shape

(2612,) float32
(2612,) float32
(2612,) float32
(2612,) float64
(2612,) float64


(2612, 5)

In [14]:
# Edge dynamic features
with h5py.File(hec_result_path, 'r') as hec:
    vel=np.array(hec['Results']['Unsteady']['Output']['Output Blocks']['Base Output'] \
                       ['Unsteady Time Series']['2D Flow Areas']['Perimeter 1']['Face Velocity'])
print(vel.shape, vel.dtype)

df_edges = vel
df_edges.shape

(337, 2612) float32


(337, 2612)

In [15]:
# Convert to list of pytorch geometric Data objects for time series
dataset = []
for i, ts in enumerate(timesteps):
    ts_df_nodes = df_nodes[i, :][:, None]
    ts_df_edges = df_edges[i, :][:, None]
    node_features = np.concatenate([sf_nodes, ts_df_nodes], axis=1)
    edge_features = np.concatenate([sf_edges, ts_df_edges], axis=1)
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features, pos=pos)
    dataset.append(data)
print(len(dataset))
print(dataset[0])

337
Data(x=[1268, 4], edge_index=[2612, 2], edge_attr=[2612, 6], pos=[1268, 2])
